In [ ]:
!pip install youtube_transcript_api
!pip install openai
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.8 MB/

In [ ]:
import re
import json
import openai
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# import re
# import openai
# from youtube_transcript_api import YouTubeTranscriptApi
# import json

class Summarizer:
    def __init__(self, link):
        self.link = link
        self.count = 0

    def youtube_parser(self):
        reg_exp = r'^.*((youtu.be/)|(v/)|(u/\w/)|(embed/)|(watch\?))\??v?=?([^#&?]*).*'
        match = re.match(reg_exp, self.link)
        return match.group(7) if match and len(match.group(7)) == 11 else False

    def get_transcript(self):
        video_id = self.youtube_parser()
        if video_id:
            try:
                ts = YouTubeTranscriptApi.get_transcript(video_id)
                transcript = " ".join([entry['text'] for entry in ts])
                return transcript
            except Exception as e:
                return f"An error occurred: {e}"
        else:
            return "Please provide a correct YouTube URL."

    def chat_with_model(self, model="gpt-3.5-turbo", user_question=""):
        transcript = self.get_transcript()

        with open('/content/API_key.json') as f:
            data = json.load(f)

        openai.api_key = data['openai_api_key']

        messages = [{"role": "system", "content": "You are the helpful and kind assistant that answers the user's doubt"},
                    {"role": "user", "content": user_question},
                    {"role": "assistant", "content": f"Answer the user's questions from this transcript, if the user asks anything that is not related to the transcript then answer that you do no know that:\n{transcript}"}]

        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0.3,
        )

        return response.choices[0].message["content"]

def summarizer(link, user_question=""):
    summ = Summarizer(link)
    response = summ.chat_with_model(user_question=user_question)
    return response

# Example usage
video_link = 'https://www.youtube.com/watch?v=3kVWye29DdU'
user_question = "Can you explain the main points of the video?"

In [ ]:
# import gradio as gr

def summarizer(link, user_question):
    summ = Summarizer(link)
    response = summ.chat_with_model(user_question=user_question)
    return response

iface = gr.Interface(
    fn=summarizer,
    inputs=[
        gr.inputs.Textbox(label="YouTube Video URL"),
        gr.inputs.Textbox(label="Your Question")
    ],
    outputs=gr.outputs.Textbox(label="Summary"),
    live=True,
    layout="vertical"
)

iface.launch(debug=True, share=False)


<ipython-input-5-b5a7d4d929cd>:11: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-5-b5a7d4d929cd>:11: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-5-b5a7d4d929cd>:11: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="YouTube Video URL"),
<ipython-input-5-b5a7d4d929cd>:12: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Your Question")
<ipython-input-5-b5a7d4d929cd>:12: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Your Question")
<ipython-input-5-b5a7d4d929cd>:12

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
